# Домашняя работа №2, часть 1 (Елизавета Клыкова)#
В первую очередь вызовем проверку на PEP-8, затем импортируем все модули, которые пригодятся нам в работе.

In [1]:
# ! pip install --user --upgrade pip
# ! pip install pycodestyle flake8 pycodestyle_magic
%load_ext pycodestyle_magic
%pycodestyle_on
# ! pip install pymorphy2
# ! pip install pymystem3

In [2]:
import collections
from collections import OrderedDict
import json
import os
import subprocess
import re
import string
from string import punctuation
from pprint import pprint
from pymorphy2 import MorphAnalyzer
# from pymystem3 import Mystem
import nltk
# nltk.download('punkt')
from nltk.tokenize import word_tokenize
# m = Mystem()
morph = MorphAnalyzer()

## Пункт 1 ##
Для работы был выбран роман Е.И. Замятина "Мы". Файл с текстом называется "hw2_txt.txt"; он должен быть расположен в одной директории с данной программой. Считаем файл и запишем его содержимое в переменную text.

In [3]:
with open("hw2_txt.txt", encoding="utf-8") as f:
    text = f.read()

## Пункт 2 ##
Обработаем текст с помощью консольного Mystem (mystem.exe, файл с программой и файл с исходным текстом должны лежать в одной папке! Там же окажется созданный файл). Распарсим текст и запишем результат в файл "mystem.json" (вместе со знаками препинания и пробелоподобными символами, каждый объект на новой строке).

In [4]:
os.system(r".\mystem.exe -c -n -i --format json hw2_txt.txt mystem.json")

0

**Примечание.** А вот так можно сделать то же самое, замерив время работы. У меня после этого перестают работать все time, расположенные ниже, и все принты. Я пыталась с этим бороться, но проиграла, и Вы просили напомнить об этом в комментариях:)

In [5]:
# %time os.system(r".\mystem.exe -c -n -i --format json hw2_txt.txt mystem.json")

1:80: E501 line too long (81 > 79 characters)


## Пункт 3 ##
Токенизируем текст с помощью NLTK:

In [6]:
tokens = word_tokenize(text)

То же самое с замером времени работы:

In [7]:
# %time tokens = word_tokenize(text)

Разберем слова с помощью Pymorphy:

In [8]:
parsed_words = [morph.parse(token) for token in tokens]

То же самое с замером времени работы:

In [9]:
# %time parsed_words = [morph.parse(token) for token in tokens]

Теперь сохраним полученный разбор в json. Удобнее всего было бы создать словарь, в котором ключами являлись бы отдельные словоформы, а значениями - варианты их разбора (тоже в виде вложенных словарей). Но в словаре не может быть одинаковых ключей, а в тексте со 100%-ной вероятностью некоторые словоформы встретятся более одного раза. Тогда можно создать список словарей, каждый из которых содержит ключ - слово и значение - множество вариантов разбора. В итоге мы получим структуру такого вида:

[{
  
	"словоформа": {
		"вариант разбора n": {
			"tags": {"full_tags": "все теги",
                     "POS": "часть речи",
                      ...
                     "voice": "залог"
            },
			"normal_form": "начальная форма",
			"score": "вероятность"
		}
        {
        "вариант разбора n+1": {
			"tags": {"full_tags": "все теги",
                     "POS": "часть речи",
                      ...
                     "voice": "залог"
            },
			"normal_form": "начальная форма",
			"score": "вероятность"
		}
	}
}]

In [10]:
parsed_json = []
for word in parsed_words:
    json_element = OrderedDict()
    parsed_word = OrderedDict()
    for number, option in enumerate(word):
        opt_num = "option_" + str(number + 1)
        parsed_option = OrderedDict()
        tags = OrderedDict()
        if "PNCT" in str(option.tag):
            tags["POS"] = "PNCT"
        else:
            tags["full_tags"] = str(option.tag)
            tags["POS"] = str(option.tag.POS)
            tags["animacy"] = str(option.tag.animacy)
            tags["aspect"] = str(option.tag.aspect)
            tags["case"] = str(option.tag.case)
            tags["gender"] = str(option.tag.gender)
            tags["involvement"] = str(option.tag.involvement)
            tags["mood"] = str(option.tag.mood)
            tags["number"] = str(option.tag.number)
            tags["person"] = str(option.tag.person)
            tags["tense"] = str(option.tag.tense)
            tags["transitivity"] = str(option.tag.transitivity)
            tags["voice"] = str(option.tag.voice)
        parsed_option["tags"] = tags
        parsed_option["normal_form"] = str(option.normal_form)
        parsed_option["score"] = str(option.score)
        parsed_word[opt_num] = parsed_option
        word_form = str(option.word)
    json_element[word_form] = parsed_word
    parsed_json.append(json_element)

Запишем полученный список словарей в новый файл "pymorphy.json" с помощью json.dump. Поскольку файл получается больше 25Мб и через Upload files в браузере его загрузить нельзя, вот ссылка на Google-диске: https://drive.google.com/file/d/1rHMYOHwQ5yJjjv637B8LOj7b9u3a7y53/view?usp=sharing. А еще архив с этим файлом тоже лежит в репозитории.

In [11]:
with open("pymorphy.json", "w", encoding="utf-8") as f:
    json.dump(parsed_json, f, ensure_ascii=False)

## Пункт 4 ##
#### Вопрос 1 ###
Выясним, какую долю слов составляет каждая часть речи в данном тексте. Чтобы найти общее число слов в тексте, придется взять не длину списка parsed_json (поскольку там присутствуют и знаки препинания тоже), а количество всех словоформ, у которых значением POS является не PNCT.

Для каждой словоформы узнаем часть речи (будем рассматривать только первый, наиболее вероятный вариант разбора). Соберем все полученные части речи (кроме PNCT) в список parts_of_speech и создадим на его основе частотный словарь pos_counter.

Заодно создадим список всех лемм слов (элементов, не являющихся знаками препинания) для пункта 5.

In [12]:
parts_of_speech = []
word_lemmas = []
for element in parsed_json:
    word = (list(element.keys()))[0]
    part_of_speech = element[word]["option_1"]["tags"]["POS"]
    if part_of_speech != "PNCT":
        parts_of_speech.append(part_of_speech)
        word_lemmas.append(element[word]["option_1"]["normal_form"])
word_amount = len(parts_of_speech)  # число собственно слов
pos_counter = collections.Counter(parts_of_speech)

Отсортируем словарь pos_counter по значениям в обратном порядке, вычислим долю каждой части речи в данном тексте и выведем полученную информацию.

In [13]:
parts = []
for key in sorted(pos_counter, key=pos_counter.get, reverse=True):
    part = round(pos_counter[key] / word_amount, 3)
    some_POS = str(key) + ": " + str(part)
    parts.append(some_POS)
print("Доля разных частей речи в данном тексте:")
for pos in parts:
    print(pos)

Доля разных частей речи в данном тексте:
NOUN: 0.209
ADJF: 0.134
VERB: 0.11
CONJ: 0.103
NPRO: 0.1
PREP: 0.096
ADVB: 0.084
PRCL: 0.067
None: 0.028
INFN: 0.019
PRTF: 0.013
ADJS: 0.01
PRED: 0.007
COMP: 0.006
GRND: 0.005
NUMR: 0.004
PRTS: 0.004
INTJ: 0.002


#### Вопрос 2 ####
Теперь найдем топ-20 по частотности глаголов и наречий. Для этого сначала запишем все начальные формы глаголов в список common_verbs, а наречий - в common_adverbs.

In [14]:
common_verbs = []
common_adverbs = []
for element in parsed_json:
    word = (list(element.keys()))[0]
    part_of_speech = element[word]["option_1"]["tags"]["POS"]
    if part_of_speech == "VERB":
        common_verbs.append(element[word]["option_1"]["normal_form"])
    elif part_of_speech == "ADVB":
        common_adverbs.append(element[word]["option_1"]["normal_form"])

Затем на основе каждого из списков создадим частотный словарь, отсортируем по значениям в обратном порядке и выведем первые 20 элементов.

In [15]:
common_v = collections.Counter(common_verbs)
common_adv = collections.Counter(common_adverbs)

print("Топ-20 наиболее частотных глаголов:")
for i, verb in enumerate(sorted(common_v, key=common_v.get, reverse=True)):
    if i < 20:
        print(verb, "-", common_v[verb], "вхождений")

print("\nТоп-20 наиболее частотных наречий:")
for j, adv in enumerate(sorted(common_adv, key=common_adv.get, reverse=True)):
    if j < 20:
        print(adv, "-", common_adv[adv], "вхождений")

Топ-20 наиболее частотных глаголов:
быть - 442 вхождений
мочь - 226 вхождений
знать - 161 вхождений
видеть - 114 вхождений
сказать - 80 вхождений
понимать - 78 вхождений
хотеть - 76 вхождений
говорить - 64 вхождений
чувствовать - 55 вхождений
увидеть - 54 вхождений
идти - 51 вхождений
слышать - 47 вхождений
помнить - 46 вхождений
думать - 39 вхождений
стать - 39 вхождений
понять - 31 вхождений
смотреть - 29 вхождений
сидеть - 23 вхождений
нумереть - 22 вхождений
остановиться - 22 вхождений

Топ-20 наиболее частотных наречий:
уже - 146 вхождений
там - 138 вхождений
сейчас - 130 вхождений
ясно - 82 вхождений
вдруг - 80 вхождений
потому - 77 вхождений
теперь - 64 вхождений
потом - 62 вхождений
здесь - 56 вхождений
очень - 56 вхождений
вниз - 54 вхождений
опять - 51 вхождений
медленно - 50 вхождений
сегодня - 49 вхождений
тут - 49 вхождений
тогда - 48 вхождений
где - 45 вхождений
совершенно - 45 вхождений
всего - 44 вхождений
вверх - 44 вхождений


## Пункт 5 ##
Используем список лемм (word_lemmas), полученный при работе Pymorphy (т.к. в NLTK нет лемматизации для русского языка). С помощью nltk.bigrams() и nltk.trigrams() создадим списки биграмм и триграмм соответственно. Затем сформируем частотный словарь по каждому из списков, отсортируем по значениям в обратном порядке и выведем топ-25 элементов.

Чтобы не выполнять одно и то же действие дважды, объявим функцию ngrams и применим ее сначала к списку биграмм, затем к списку триграмм.

In [16]:
def ngrams(ngram_list):
    ngram_dict = collections.Counter(ngram_list)
    top25_list = []
    for i, gram in enumerate(sorted(ngram_dict, key=ngram_dict.get,
                                    reverse=True)):
        if i < 25:
            line = str(gram) + " - " + str(ngram_dict[gram]) + " вхождений"
            top25_list.append(line)
    return top25_list

In [17]:
bigram_list = list(nltk.bigrams(word_lemmas))
trigram_list = list(nltk.trigrams(word_lemmas))

top_bigrams = ngrams(bigram_list)
print("Топ-25 биграмм:")
for gram in top_bigrams:
    print(gram)

top_trigrams = ngrams(trigram_list)
print("\nТоп-25 триграмм:")
for gram in top_trigrams:
    print(gram)

Топ-25 биграмм:
('и', 'я') - 134 вхождений
('я', 'не') - 110 вхождений
('у', 'я') - 92 вхождений
('мочь', 'быть') - 91 вхождений
('не', 'мочь') - 80 вхождений
('что', 'я') - 71 вхождений
('не', 'знать') - 71 вхождений
('это', 'быть') - 69 вхождений
('и', 'весь') - 66 вхождений
('потому', 'что') - 63 вхождений
('и', 'вот') - 61 вхождений
('весь', 'это') - 60 вхождений
('я', 'быть') - 59 вхождений
('в', 'я') - 55 вхождений
('единый', 'государство') - 49 вхождений
('и', 'не') - 48 вхождений
('я', 'в') - 47 вхождений
('если', 'бы') - 46 вхождений
('не', 'быть') - 45 вхождений
('я', 'и') - 45 вхождений
('я', 'видеть') - 44 вхождений
('я', 'знать') - 43 вхождений
('и', 'в') - 43 вхождений
('так', 'же') - 41 вхождений
('на', 'я') - 40 вхождений

Топ-25 триграмм:
('я', 'не', 'мочь') - 23 вхождений
('один', 'и', 'тот') - 13 вхождений
('и', 'тот', 'же') - 13 вхождений
('я', 'не', 'знать') - 13 вхождений
('так', 'же', 'как') - 12 вхождений
('в', 'самый', 'дело') - 12 вхождений
('и', 'тотчас', 'же

Почему же биграммы и триграммы получились именно такими? С одной стороны, в топ-25 попали сочетания многих очень частотных слов (тех самых, которые являются стоп-словами в NLTK). С другой стороны, некоторые сочетания отражают тематику выбранного текста ("единый государство", "до сей пора", "в древний дом", "там за стена" и др.)